# ITRLM+RAG Exploration Notebook

This notebook provides exploratory analysis and testing of the ITRLM+RAG implementation.

## Overview
- **ITRLM**: Improved Translation-based Language Model
- **RAG**: Retrieval-Augmented Generation
- **Hybrid Ranking**: Multi-stage ranking pipeline


In [1]:
# Import necessary libraries
import sys
sys.path.append('..')

from data.semeval_loader import SemEvalLoader
from data.yahoo_loader import YahooLoader
from hmr.text_processing import TextProcessor
from hmr.pmi_dictionary import PMIDictionary
from hmr.category_predictor import CategoryPredictor
from hmr.rag_generator import RAGAnswerGen

/Users/ayush/.pyenv/versions/3.10.14/envs/itrlm-rag-310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ------------------------------
# 1. Load and prepare data
# ------------------------------
semeval_loader = SemEvalLoader('../data')
yahoo_loader = YahooLoader('../data')

test_data = semeval_loader.load_data('test')
train_data = yahoo_loader.load_data('train', max_samples=100)

print(f"Loaded {len(test_data)} test examples")
print(f"Loaded {len(train_data)} training examples")

qa_pairs = [(ex['question'], ex['answer']) for ex in test_data]
processor = TextProcessor()
qa_pairs_clean = [(processor.clean(q), processor.clean(a)) for q, a in qa_pairs]


Test file not found: ../data/semeval_2017/test_data.txt
Train file not found: ../data/yahoo_answers/yahoo_answers_train.json


Loaded 3 test examples
Loaded 5 training examples


## 1. Data Loading and Exploration


In [3]:
# ------------------------------
# 2. Build PMI dictionary
# ------------------------------
pmi_builder = PMIDictionary({"alpha": 0.3, "top_n_translations": 50})
pmi_builder.build(qa_pairs_clean)

[PMI] Building general dictionary from 3 Q-A pairs...


PMI calculation: 100%|██████████| 46/46 [00:00<00:00, 505073.26it/s]

✅ PMI dictionary saved to outputs/pmi_dicts/pmi_general.json


In [4]:
# ------------------------------
# 3. Category prediction
# ------------------------------
yahoo_pred = CategoryPredictor()
yahoo_pred.load_or_train()
cat, conf = yahoo_pred.predict("Where can I buy cheap airline tickets?")
print(f"Predicted Category: {cat} (Confidence: {conf:.2f})")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Loaded Category Predictor from outputs/checkpoints/bert_category_predictor/model.pt (device: mps)
Predicted Category: Business & Finance (Confidence: 0.67)


In [ ]:
# ------------------------------
# 4. RAG setup
# ------------------------------
cfg_rag = {
    "embed_model": "sentence-transformers/all-mpnet-base-v2",
    "gen_model": "google/flan-t5-base", # "mistralai/Mistral-7B-Instruct-v0.2",   # << bigger model
    "top_k_ctx": 5,
    "max_answer_tokens": 50
}

rag_gen = RAGAnswerGen(cfg_rag)


/Users/ayush/.pyenv/versions/3.10.14/envs/itrlm-rag-310/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[RAG] Using device: mps
[RAG] Using Seq2SeqLM model: google/flan-t5-base


In [6]:
# ------------------------------
# 5. Build or attach corpus safely
# ------------------------------
if hasattr(rag_gen, "build_or_load_index"):
    rag_gen.build_or_load_index(qa_pairs_clean)
    print("Index built using build_or_load_index()")
elif hasattr(rag_gen, "fit"):
    rag_gen.fit(qa_pairs_clean)
    print("Index built using fit()")
elif hasattr(rag_gen, "add_corpus"):
    rag_gen.add_corpus(qa_pairs_clean)
    print("Corpus added using add_corpus()")
else:
    print("⚠️ No index build method found — make sure retrieve_context() uses default data")

⚠️ No index build method found — make sure retrieve_context() uses default data


In [7]:
# ------------------------------
# 6. Single Query Demo
# ------------------------------
query = "Where can I buy cheap airline tickets?"
answer = rag_gen.generate(query)

print("\nSingle Query Test:")
print(f"Q: {query}")
print(f"A: {answer}")

✅ Loaded FAISS index from outputs/faiss_index.index
✅ Loaded 2 context texts from outputs/faiss_index_texts.json

Single Query Test:
Q: Where can I buy cheap airline tickets?
A: You can buy tickets on airline websites.


In [9]:
# ------------------------------
# 7. Batch Generation & Evaluation
# ------------------------------
test_questions = [
    "Where can I buy cheap airline tickets?",
    "What is the best way to book a flight online?",
    "Which website offers discounts on airfare?",
    "How do I find cheap tickets?",
    "Where to purchase airline tickets?"
]

# Generate answers
predicted_answers = [rag_gen.generate(q) for q in test_questions]

# Mock reference answers
reference_answers = [
    "You can buy tickets on airline websites.",
    "Use travel aggregators like Skyscanner.",
    "You can buy tickets on airline websites.",
    "Use travel aggregators like Skyscanner.",
    "You can buy tickets on airline websites.",
]

# Simple evaluation
correct = sum([pred.strip().lower() == ref.strip().lower()
               for pred, ref in zip(predicted_answers, reference_answers)])
accuracy = correct / len(test_questions)

print("\nBatch Evaluation:")
for q, pred, ref in zip(test_questions, predicted_answers, reference_answers):
    print(f"Q: {q}\n  Predicted: {pred}\n  Reference: {ref}\n")

print(f"Accuracy: {accuracy:.2f}")
print(f"Last generated answer: {answer}")


Batch Evaluation:
Q: Where can I buy cheap airline tickets?
  Predicted: You can buy tickets on airline websites.
  Reference: You can buy tickets on airline websites.

Q: What is the best way to book a flight online?
  Predicted: You can buy tickets on airline websites.
  Reference: Use travel aggregators like Skyscanner.

Q: Which website offers discounts on airfare?
  Predicted: You can buy tickets on airline websites.
  Reference: You can buy tickets on airline websites.

Q: How do I find cheap tickets?
  Predicted: You can buy tickets on airline websites. Use travel aggregators like Skyscanner. Use travel aggregators like Skyscanner.
  Reference: Use travel aggregators like Skyscanner.

Q: Where to purchase airline tickets?
  Predicted: You can buy tickets on airline websites.
  Reference: You can buy tickets on airline websites.

Accuracy: 0.60
Last generated answer: You can buy tickets on airline websites.
